# Threshold Analysis

In [1]:
import os

# user defined methods
import sys
sys.path.append('../utils')

from threshold_logs import extract_info_folder, extract_log

log_folders = os.listdir('../../results/threshold/2023-11-10/')


In [2]:
print(len(log_folders))

720


In [3]:
best_false_negative = 1000
for folder in log_folders:
    f1_test, f1_val, cm = extract_log('../../results/threshold/2023-11-10/' + folder + '/find_threshold.log')

    if f1_test is None:
        continue
    # find the lowest false negative rate
    false_negative = cm[0][1]
    if false_negative < best_false_negative:
        best_false_negative = false_negative
        best_folder = folder
        best_cm = cm
        best_f1_test = f1_test
        best_f1_val = f1_val

In [4]:
extract_info_folder(best_folder)

Dataset: feature_selection_mrmr20
Model: decision_tree
Target: hf15
Threshold CAHI > 2
Threshold CAHI > OAHI * 1/1
